## Notebook for testing plots of the number concentration

In [28]:
# imports from packages

#import pandas as pd
import xarray as xr
#import numpy as np
import matplotlib.pyplot as plt
#from matplotlib.gridspec import GridSpec # gridspec for nested subfigures
import matplotlib.dates as mdates
#import cartopy.crs as ccrs
#import cartopy.feature as creature
import glob
import os
#import sys # to test parts of code with sys.exit()
#import math
#import seaborn as sns

#import functions
#import numb_conc # functions relating to number concenctration calculations
#import plots


In [29]:
# --- Global formatting/settings/variables

sample_rate = 1 # alternatives: 1 s, 5 s, 12 s.

# formatting for only showing time on the x-axis for plots per flight
# Use by setting axs[1,0].xaxis.set_major_formatter(formatter) for each axis where only time should show
formatter = mdates.DateFormatter('%H:%M')

In [30]:
def preselect_ds(ds, remove_cirrus_T = -30, marine_lat = 70):
    """Function to remove cirrus (from temperature) and land values (from latitude)

    Parameters
    ----------
    ds
        xarray dataset with all microphy values
    save_path
        str, path to where plots should be saved
    remove_cirrus_T
        Temperature, default = -30. Remove cirrus by only using values above this temperature threshold
        Everything is included if set to ""
    marine_lat
        Latitude value to include. only latitudes above this threshold is included.
        Everything is included if set to ""
    
    Returns
    -------
    presel_ds
        xarray dataset with only the preselected values
    presel_info_txt
        list of text for plots etc: [0]-> short descr (for filenames) [1]-> long description
    """

    short_txt = ''
    long_txt = ''

    # masks for preselection
    if marine_lat != "":
        marinelat_mask = (ds['lat']>=marine_lat).compute()
        ds = ds.where(marinelat_mask, drop = True)
        short_txt = short_txt + f'_lat{marine_lat}'
        long_txt = long_txt + f' Lat<{marine_lat} removed, '
    else:
        short_txt = short_txt + f'_latall'
        long_txt = long_txt + f'All lat included, '

    if remove_cirrus_T != "":
        cirrusT_mask = (ds['T']>=remove_cirrus_T).compute()
        ds = ds.where(cirrusT_mask, drop = True)
        short_txt = short_txt + f'_T{remove_cirrus_T}'
        long_txt = long_txt + f' Temp<{remove_cirrus_T}C removed, '
    else:
        short_txt = short_txt + f'_Tall'
        long_txt = long_txt + f'All T included, '
    
    # Define presel_info_txt
    presel_info_txt = [short_txt, long_txt] 

    return ds, presel_info_txt


#preselect_ds(1)

In [31]:
def sel_incloud_values(ds, th_method = 'LWC_IWC_th'):
    """Function to select out dataset that only includes incloud values according to settings

    Parameters
    ----------
    ds
        xarray dataset with all microphy values
    save_path
        str, path to where plots should be saved
    th_method
        str, which method should be used for determining incloud values
            - 'LWC_IWC_th' based on LWC and IWC (Default)
            - 'LWC_th' based on only LWC
            - 'TWC_th' based on only TWC
            - 'N_th' based on number concentration from both CDP and CIP


    Returns
    -------
    incloud_ds
        xarray dataset with only the incloud values
    incloud_info
        long description of threshold used (for plots etc.)
    """

    # Define masking of ds based on th_method selection
    if th_method == 'LWC_IWC_th':
        # ----- Water content threshold
        # following the korolev 22 paper: "In the present study the thresholds for liquid water content and ice water content (IWC) 
        # were set as LWC > 0.01 g m−3, IWC > 0.01 g m−3, respectively. The phase composition of clouds was identified based on the 
        # assessment of the ice water fraction mu = IWC∕(LWC + IWC). Thus, clouds with mu=0.9 were considered as ice, 
        # clouds with Mu =0.1 were defined as liquid, 
        # and clouds 0.1 ≤ 𝜇𝜇 ≤ 0.9 were determined as mixed-phase clouds."
        
        lwc_th = 0.01
        
        # either lwc or iwc needs to be larger than the threshold, use lwc_iwc_mask
        incloud_mask = ((ds['LWC corr']>= lwc_th)|(ds['IWC100']>= lwc_th)).compute() # mask the values based on lwc or iwc according to threshold

        long_txt = f'{lwc_th} m^-3, (LWC or IWC)'

    elif th_method == 'LWC_th':
        # only lwc have to be larger than threshold, use lwc mask
        lwc_th = 0.01

        incloud_mask = (ds['LWC corr']>=lwc_th).compute() # mask the values based on lwc according to threshold
        long_txt = f'{lwc_th} m^-3, (LWC)'

    elif th_method == 'TWC_th':
        # twc have to be larger than threshold value, use twc mask
        lwc_th = 0.01

        incloud_mask = (ds['TWC']>=lwc_th).compute() # mask the values based on twc according to threshold
        long_txt = f'{lwc_th} m^-3, (TWC)'

    elif th_method == 'N_th':
        # ----- Number concentration threshold
        # Following table 2 from Evans et al 2025:
        # Ice concentration threshold to define ice = 0.1 L-1 (or m-3) (NT100 is given in m-3)
        # Cdp drop concentration to define liquid = 2 cm-3 (numb conc corrected is given in cm-3)

        n_ice_th = 0.1
        n_drp_th = 2
        
        incloud_mask = ((ds['Number Conc corr']>= n_drp_th)|(ds['NT100']>= n_ice_th)).compute()

        long_txt = f'Nt_cdp>{n_drp_th} cm^-3, Nt_cip100>{n_ice_th} L^-1'

    else:
        print('WARNING: in-cloud threshold method not defined!')
        return

    # create the selected dataset based on selected mask
    incloud_ds = ds.where(incloud_mask, drop = True)

    incloud_info = [th_method, long_txt]
    return incloud_ds, incloud_info



In [32]:
# --- Data import

main_path = '/home/ninalar/Documents/MC2/Results_2022-islas/Processed/ISLAS_processed' # regular path
file_struct = f'/microphy_{sample_rate}s*.nc' # structure of cip text-file names

# get all the .nc files in the main path
files = glob.glob(main_path + file_struct)

# Exclude the file containing 'IS22-09' (flew over land)
files_to_exclude = [f for f in files if 'IS22-09' in os.path.basename(f)]
files_to_include = [f for f in files if f not in files_to_exclude]


ds = xr.open_mfdataset(files_to_include, combine='by_coords', combine_attrs='drop_conflicts') # drop the IS22-09 flight

In [ ]:
# Main selection of data
pre_ds, info = preselect_ds(ds) # do preselection with default values (T>-30, lat>70)
incloud_ds, incloud_desc = sel_incloud_values(pre_ds) # select incloud values with default method (LWC_TWC_th)

# Get short and long descriptions
short_desc = info[0]+'_'+ incloud_desc[0]
long_desc = info[1]+incloud_desc[1]

_lat70_T-30_LWC_IWC_th
 Lat<70 removed,  Temp<-30C removed, 0.01 m^-3, (LWC or IWC)


In [34]:
incloud_desc

['LWC_IWC_th', '0.01 m^-3, (LWC or IWC)']

In [ ]:
def numb_conc_plot_same(ax, ds_n, ds_s, cat_text):
    import numb_conc # number concentration functions

    #handles and labels for legends
    h_hist = []
    l_hist = []
    
    # Get the normalized values for plotting (from functions in numb_conc)
    n_cip_part_norm = numb_conc.cip_mean_norm_Nt(ds_n)
    n_cdp_part_norm = numb_conc.cdp_mean_norm_Nt(ds_n)
    s_cip_part_norm = numb_conc.cip_mean_norm_Nt(ds_s)
    s_cdp_part_norm = numb_conc.cdp_mean_norm_Nt(ds_s)

    # plot histogram based on normalized data and binning size information for CDP and CIP
    # North
    # exclude the 4 first bins of cdp (lower than 6 microns)
    h0  = ax.hist(ds_n['Size'].values[:3], weights = n_cdp_part_norm.values[:3], bins=ds_n['Size'].values[:3], label = "CDP north", 
            histtype='step', alpha = 0.7, color='grey')
    h1  = ax.hist(ds_n['Size'].values[3:], weights = n_cdp_part_norm.values[3:], bins=ds_n['Size'].values[3:], label = "CDP north", 
            histtype='step', alpha = 0.7, linewidth = 2, color='deepskyblue')
    
    h_hist.append(h1[2][0])
    l_hist.append('CDP North')
    
    midbins = ds_n['MIDBINS'].values
    weights = n_cip_part_norm.values
    h2 = ax.hist(midbins[:3], weights=weights[:3], bins=midbins[:3], label="CIP North", color='grey', histtype='step', alpha=0.3)# Plot first three bins in grey
    h3 = ax.hist(midbins[3:], weights=weights[3:], bins=midbins[3:], label="CIP North", color='lightskyblue', linewidth = 2, histtype='step', alpha = 0.5)
    h_hist.append(h3[2][0])
    l_hist.append('CIP North')
    
    # --- calculate mean number concentration from the histogram 
    # - CDP
    prod_val_count_cdp = ds_n['Size'].values*n_cdp_part_norm.values
    tot_sum = prod_val_count_cdp[3:].sum()
    counts= n_cdp_part_norm.values[3:]
    count_sum = counts.sum()
    mean_size_cdp = tot_sum/count_sum
    # set as vertical line in plot
    l1 = ax.axvline(mean_size_cdp, color='deepskyblue', linestyle='dashed', linewidth=2, label=f'CDP North: {mean_size_cdp:.2f} $\mu$m')
    
    # - CIP
    # Only calculate the mean on the bins that are not greyed out (the first 3 bins)
    prod_val_count_cip = n_cip_part_norm.values*ds_n['MIDBINS'].values
    tot_sum = prod_val_count_cip[3:].sum()
    count_sum = n_cip_part_norm.values[3:].sum()
    mean_size_cip = tot_sum/count_sum
    # set as vertical line in plot
    l2 = ax.axvline(mean_size_cip, color='lightskyblue', linestyle='dashed', linewidth=2, label=f'CIP North: {mean_size_cip:.2f} $\mu$m')

    # South
    h4 = ax.hist(ds_s['Size'].values[3:], weights = s_cdp_part_norm.values[3:], bins=ds_s['Size'].values[3:], label = "CDP South", 
            histtype='step', alpha = 0.7,linewidth = 2, color = 'red')
    h7 = ax.hist(ds_s['Size'].values[:3], weights = s_cdp_part_norm.values[:3], bins=ds_s['Size'].values[:3], label = "CDP South", 
            histtype='step', alpha = 0.7, color = 'grey')
    h_hist.append(h4[2][0])
    l_hist.append('CDP South')
    
    midbins = ds_s['MIDBINS'].values
    weights = s_cip_part_norm.values
    h5 = ax.hist(midbins[:3], weights=weights[:3], bins=midbins[:4], label="CIP South", color='grey', histtype='step', alpha=0.3)# Plot first three bins in grey
    h6 = ax.hist(midbins[3:], weights=weights[3:], bins=midbins[3:], label="CIP South", color = 'salmon', linewidth = 2, histtype='step', alpha = 0.5)
    h_hist.append(h6[2][0])
    l_hist.append('CIP South')
    h_hist.append(h5[2][0])
    l_hist.append('ignore 3 first bins \n of CIP and CDP)')
    
    # --- calculate mean number concentration from the histogram
    # - CDP
    prod_val_count_cdp = ds_s['Size'].values*s_cdp_part_norm.values
    tot_sum = prod_val_count_cdp[3:].sum()
    counts= s_cdp_part_norm.values[3:]
    count_sum = counts.sum()
    mean_size_cdp = tot_sum/count_sum
    # set as vertical line in plot
    l3 = ax.axvline(mean_size_cdp, color='red', linestyle='dashed', linewidth=2, label=f'CDP South: {mean_size_cdp:.2f} $\mu$m')
    
    # - CIP
    # Only calculate the mean on the bins that are not greyed out (the first 3 bins)
    prod_val_count_cip = s_cip_part_norm.values*ds_s['MIDBINS'].values
    tot_sum = prod_val_count_cip[3:].sum()
    count_sum = s_cip_part_norm.values[3:].sum()
    mean_size_cip = tot_sum/count_sum
    # set as vertical line in plot
    l4 = ax.axvline(mean_size_cip, color='salmon', linestyle='dashed', linewidth=2, label=f'CIP South: {mean_size_cip:.2f} $\mu$m')
   
    
    # set labels, titles and legends for subplots
    ax.set_ylabel('dN/dlogDp (#/$m^4$)', fontsize=16)
    ax.set_xlabel('Size ($\mu$m)',fontsize = 16)
    ax.set_title(cat_text, fontsize=18)

    # first legend:
    legend1 = ax.legend(handles=h_hist,labels = l_hist, loc='upper right', title='Size distribution', fontsize = 12, title_fontsize=12)
    ax.add_artist(legend1)

    #second legend:
    legend2 = ax.legend(handles=[l1,l2,l3,l4], loc = 'center right', title='Mean size', fontsize = 12,title_fontsize=12)
    ax.add_artist(legend2)

In [ ]:
##### --- Plotting number concentrations in same plot (top and bulk)
### removing the values of cip that is below 6 um

fig, ax = plt.subplots(2, 1, figsize=(11, 13), layout="constrained", sharey=True)

# Top layer
numb_conc_plot_same(ax[0], north_top_ds, south_top_ds, f'Top layer of cloud (N:{n_top_list[0]}m, S:{s_top_list[0]}m)')

# Bulk layers
numb_conc_plot_same(ax[1], north_bulk_ds, south_bulk_ds, f'Bulk of cloud (N:{n_bulk_list[0]}-{n_bulk_list[-1]}m, S:{s_bulk_list[0]}-{s_bulk_list[-1]}m)')


# --- Subplot settings
for ax in ax.ravel():  # Flatten the array of axes and iterate
    ax.set_xscale('log')  # Set x-axis to log scale
    ax.set_yscale('log')  # Set y-axis to log scale
    ax.grid(True)
    #ax.label_outer()  # This method hides only labels in the inner axes; keep it commented out to show labels
    ax.yaxis.set_tick_params(which='both', labelleft=True)
    ax.xaxis.set_tick_params(which='both', labelbottom=True)

# --- Figure 
#plt.suptitle(f'Number concentration in 300m altitude layers per region (North marine, South marine) \n \
#            Pre-selection: {pre_text}\n \
#            in-cloud threshold method: {th_method}, threshold(s): {th}') # title
plt.suptitle(f'Number concentration in 300m altitude layers \n per region (Northern marine, Southern marine)', fontsize=20)
plt.savefig(save_path+f'{th_method}/Ntjoint_Latitude_bands_2layer_rem3bins{th_method}{preopt}.png') # Save to
plt.show()